# Strategising Template-guided Needle Placement for MR-targeted Prostate Biopsy

## Reinforcement Learning
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science<br>

```
Gayo, I. J., Saeed, S. U., Barratt, D. C., Clarkson, M. J., & Hu, Y. (2022). Strategising template-guided needle placement for MR-targeted prostate biopsy. In MICCAI Workshop on Cancer Prevention through Early Detection (pp. 149-158). Springer, Cham.
```

<center>
<img src = "figures/model.JPG" width = "1250"/> <br>
</center>

##### <code>Objective</code>: Investigate the feasibility of using Reinforcement Learning (RL) to plan the patient-specific needle sampling strategies, optimized in pre-operative magnetic resonance (MR)-derived RL environments. 

##### <code>Environment</code>: The authors developed an environment for template-guided biopsy sampling of the cancer targets. The environment is a 2D-slice of an MR-derived biopsy environment where virtual biopsy needles can be inserted through the perineum via a brachytherapy template gtid consisting of $13 \times 13$ holes that are 5 mm apart. 

The following points are certain considerations the authors assumed in designing and constructing the adopted biopsy environment.

1. The prostate gland from each MR volume, the MR-identified targets, and key landmarks such as the position of the rectum are all **manually segmented from indivudal patients to construct the biopsy environment**. 
2. Binary segmentation are provided as observations for the RL agents, as opposed to ultrasound image intensities, which are neither available during planning nor straightforward to synthesize from MR images. 
3. Uncertainty in MR-to-Ultrasound registration can be added to the segmented regions, together with other potential erros in localizing these errors during observation nsuch as observer variability in manual segmentation. 

##### <code>State</code>: At a given time point $t$ during the procedure, the agent receives information about its current state $s_t ∈ S$ : the chosen grid point and the re-sampled 2D image plane obtained by rotating the probe to the current template grid position.

## **RADI608: Data Mining and Machine Learning**

### Assignment: Reinforcement Learning
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

Note: In case of Python Markdown errors, you may access the assignment through this GitHub [Link](https://github.com/rrwabina/RADI608/tree/main/Submitted)